# **Dự án phân tích dữ liệu thương mại điện tử - Phân tích dữ liệu**

## Mục tiêu
- Phân tích dữ liệu đã làm sạch để tìm insight về doanh thu, sản phẩm bán chạy, và hành vi khách hàng.
- Chuẩn bị dữ liệu cho trực quan hóa và báo cáo.

## Dữ liệu đầu vào
- `cleaned_orders.csv`: Thông tin đơn hàng đã làm sạch (~99,441 dòng).
- `cleaned_order_items.csv`: Chi tiết sản phẩm đã làm sạch (~112,650 dòng).

In [7]:
# Import thư viện cần thiết
import pandas as pd
import numpy as np

# Đọc file CSV đã làm sạch
orders = pd.read_csv('D:/Ecommerce_Analysis/data/cleaned_data/cleaned_orders.csv')
order_items = pd.read_csv('D:/Ecommerce_Analysis/data/cleaned_data/cleaned_order_items.csv')
products = pd.read_csv('D:/Ecommerce_Analysis/data/cleaned_data/cleaned_products_items.csv')



# Xác nhận kiểu dữ liệu và kích thước
print("Kiểu dữ liệu của orders:")
print(orders.dtypes)
print("\nKiểu dữ liệu của order_items:")
print(order_items.dtypes)
print("\nKích thước orders:", orders.shape)
print("Kích thước order_items:", order_items.shape)

FileNotFoundError: [Errno 2] No such file or directory: 'D:/Ecommerce_Analysis/data/cleaned_data/cleaned_products_items.csv'

In [6]:
# Vì lưu file CSV có thể mất định dạng datetime, thêm cell code để khôi phục
# Chuyển đổi lại các cột thời gian trong orders
date_columns = [
    'order_purchase_timestamp',
    'order_approved_at',
    'order_delivered_carrier_date',
    'order_delivered_customer_date',
    'order_estimated_delivery_date'
]
for col in date_columns:
    orders[col] = pd.to_datetime(orders[col], errors='coerce')

# Chuyển đổi shipping_limit_date trong order_items
order_items['shipping_limit_date'] = pd.to_datetime(order_items['shipping_limit_date'], errors='coerce')

# Kiểm tra lại
print("\nKiểu dữ liệu sau khi chuyển đổi:")
print("orders:")
print(orders.dtypes)
print("order_items:")
print(order_items.dtypes)


Kiểu dữ liệu sau khi chuyển đổi:
orders:
order_id                                 object
customer_id                              object
order_status                             object
order_purchase_timestamp         datetime64[ns]
order_approved_at                datetime64[ns]
order_delivered_carrier_date     datetime64[ns]
order_delivered_customer_date    datetime64[ns]
order_estimated_delivery_date    datetime64[ns]
dtype: object
order_items:
order_id                       object
order_item_id                   int64
product_id                     object
seller_id                      object
shipping_limit_date    datetime64[ns]
price                         float64
freight_value                 float64
dtype: object


## Câu hỏi kinh doanh (Business Questions)
1. Danh mục sản phẩm nào bán chạy nhất (doanh thu và số lượng)?
2. Chi nhánh/thành phố nào có doanh thu cao nhất?
3. Thời gian giao hàng trung bình là bao lâu?
4. Phương thức thanh toán phổ biến nhất là gì? (sẽ cần mở rộng sang `order_payments` sau)
5. Có xu hướng mua sắm theo thời gian không?

### Ghi chú
- Sẽ tập trung vào các câu hỏi 1, 2, 3 trong bước đầu.
- Có thể mở rộng sang các file khác (như `order_payments`) nếu cần.